# Vectorized Models

## Emotion Face Classifier Notebook 2

Vectorized models for 2-D image classification is simpler and less computationally expensive than more sophisticated image models (e.g., deep learning, CNNs). 

This notebook generates a numpy array from the data and applies vectorized models to the data. 

See `README.md` for more details on approach benefits and disadvantages. 

In [2]:
import os
import pandas as pd
import plotly.express as px

from skimpy import skim_get_figure

In [3]:
def check_directory_name(target_name) -> bool:
    """
    Check if the current directory name matches the target_name.
    If not, move up a directory and repeat the check.
    
    Args:
        target_name (str): The directory name to match.
        
    Returns:
        bool: True if the current directory name matches the target_name, False otherwise.
    """
    # Get the current directory path
    current_dir = os.getcwd()
    # Extract the directory name from the path
    current_dir_name = os.path.basename(current_dir)    
    # Check if the current directory name matches the target_name
    if current_dir_name == target_name:
        print(f'Directory set to {current_dir}, matches target dir sting {target_name}.')
        return True
    else:
        # Move up a directory
        os.chdir('..')
        # Check if we have reached the root directory
        if os.getcwd() == current_dir:
            return False
        # Recursively call the function to check the parent directory
        return check_directory_name(target_name)

In [4]:
main_dir = 'EmotionFaceClassifier'
check_directory_name(main_dir)

Directory set to /Users/dsl/Documents/GitHub/EmotionFaceClassifier, matches target dir sting EmotionFaceClassifier.


True

In [5]:
from utils.helpers import (
    load_config
)

from utils.data_ingest_eda import (
    generate_file_dataframe,
    emotion_count_piv,
    piv_stacked_bar,
    waffle_chart,
    show_example_images,
    convert_pixels_to_array,
    create_img
)